In [ ]:
import os
import torch
import typing
import pandas as pd
import numpy as np
from PIL import Image
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [ ]:
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

TRAIN_IMAGES_PATH = 'data/public_training_set_release_2.0/images/'
TRAIN_LABELS = 'data/public_training_set_release_2.0/annotations.json'

# Data Exploration

In [ ]:
def read_image(path: str) -> np.ndarray:
    return np.array(Image.open(path))

def show_image_coco(image_id: int, coco_labels: COCO, with_mask: bool = True) -> np.ndarray:
    im_info = coco_labels.loadImgs(image_id)[0]
    
    if not with_mask:
        plt.imshow(read_image(TRAIN_IMAGES_PATH + im_info['file_name']))
    else:
        objs_ann = labels.imgToAnns[image_id]
        
        res_image = read_image(TRAIN_IMAGES_PATH + im_info['file_name'])

        plt.imshow(res_image)
        labels.showAnns(objs_ann)

In [ ]:
labels = COCO(TRAIN_LABELS)

In [ ]:
dir(lables)

In [ ]:
img_ids = labels.getImgIds()
#184135
labels.imgToAnns[img_ids[1]]

In [ ]:
labels.imgToAnns[img_ids[1]]

In [ ]:
show_image_coco(img_ids[2], labels, True)

# Dataset

In [ ]:
class FoodDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms

        self.coco_ds = COCO(TRAIN_LABELS)
        self.img_ids  = sorted(self.coco_ds.getImgIds())
        
    def __getitem__(self, idx):

        img_path = "{}"
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)

        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
f"{3:05d}"